In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os
print(os.listdir("../input"))

**Data loading**

In [226]:
df = pd.read_csv('../input/train.csv',index_col='id')
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
#df['trip_duration'] = df['trip_duration'] / 3600

In [199]:
df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,year_pickup,month_pickup,day_pickup,hour_pickup,weekday_pickup,minute_pickup,second_pickup
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1458644.0,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02,2016.0,3.516818e+00,1.550402e+01,1.360648e+01,3.050375e+00,2.959016e+01,2.947359e+01
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03,0.0,1.681038e+00,8.703135e+00,6.399693e+00,1.954039e+00,1.732471e+01,1.731985e+01
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00,2016.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02,2016.0,2.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,1.500000e+01,1.400000e+01
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02,2016.0,4.000000e+00,1.500000e+01,1.400000e+01,3.000000e+00,3.000000e+01,2.900000e+01
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03,2016.0,5.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,4.500000e+01,4.400000e+01
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06,2016.0,6.000000e+00,3.100000e+01,2.300000e+01,6.000000e+00,5.900000e+01,5.900000e+01


**Split datetime**

In [235]:
    df_datetime = df['pickup_datetime']
    df['year_pickup'] = df_datetime.dt.year
    df['month_pickup'] = df_datetime.dt.month
    df['day_pickup'] = df_datetime.dt.day
    df['hour_pickup'] = df_datetime.dt.hour
    df['weekday_pickup'] = df_datetime.dt.weekday
    df['minute_pickup'] = df_datetime.dt.minute
    df['second_pickup'] = df_datetime.dt.second

**Filter the data**


Nous avons tout d'abord verifier l'incohérence dans les données.
Pour la durée de la course on a remarqué des incohérences (des courses qui duraient entre 3h et 10h pour le maximum) et Nous avons aussi remarqué que le nombre de passager était de 0 pour certaines courses)
On a donc décidé de filtrer ces données de la manière suivante:

In [247]:
selected_columns = ['vendor_id','passenger_count','pickup_longitude','pickup_latitude',
                    'dropoff_longitude','dropoff_latitude','year_pickup','month_pickup',
                    'day_pickup','hour_pickup','weekday_pickup','minute_pickup','second_pickup']


filter_passenger_duration =  (df['passenger_count'] > 0) & (df['trip_duration'] > 120) & (df['trip_duration'] < 10800 )
filter_df = df.loc[filter_passenger_duration]
X = filter_df[selected_columns]
y = filter_df['trip_duration']
X.shape, y.shape

((1428201, 13), (1428201,))

**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

**Create the shuffle split**

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle = ShuffleSplit(n_splits=5, test_size=0.99, random_state=42)

**DETERMINE BEST PARAMS for GridSearch
(Non utilisé)**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100,200,500,700],
    'max_features': ['auto'],
    "max_depth": [1,3,5,10]
}
gs = GridSearchCV(estimator=rf,cv=shuffle,param_grid=param_grid,scoring="neg_mean_squared_log_error")
#gs.fit(X, y)
#print(gs.best_params_)
#y_predict = gs.predict(X_test)
#print(y_predict.mean())

**CROSS VALIDATION**

In [248]:
from sklearn.model_selection import cross_val_score
import sklearn
losses = cross_val_score(rf, X, y, scoring="neg_mean_squared_log_error", cv=shuffle)
np.sqrt(- losses.mean())

0.43479862575535966

In [239]:
df_test = pd.read_csv('../input/test.csv',index_col="id")
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])
df_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
id,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


**Split DATETIME TEST**

In [240]:
df_test_datetime = df_test['pickup_datetime']
df_test['year_pickup'] = df_test_datetime.dt.year
df_test['month_pickup'] = df_test_datetime.dt.month
df_test['day_pickup'] = df_test_datetime.dt.day
df_test['hour_pickup'] = df_test_datetime.dt.hour
df_test['weekday_pickup'] = df_test_datetime.dt.weekday + 1
df_test['minute_pickup'] = df_test_datetime.dt.minute
df_test['second_pickup'] = df_test_datetime.dt.second
df_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,year_pickup,month_pickup,day_pickup,hour_pickup,weekday_pickup,minute_pickup,second_pickup
id,,,,,,,,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2016,6,30,23,4,59,58
id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2016,6,30,23,4,59,53
id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,2016,6,30,23,4,59,47
id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,2016,6,30,23,4,59,41
id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,2016,6,30,23,4,59,33


**Train the data**

In [241]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

**Predict the data**

In [242]:
X_test = df_test[selected_columns]
X_test.shape
y_pred = rf.predict(X_test)
y_pred.mean(), len(y_pred)

(838.7576905111542, 625134)

**Data submission**

In [250]:
submission = pd.read_csv('../input/sample_submission.csv') 
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


**Submission**

In [244]:
submission['trip_duration'] = y_pred
submission.to_csv('submission.csv', index=False)
submission.head()

,id,trip_duration
0,id3004672,701.9
1,id3505355,630.2
2,id1217141,713.4
3,id2150126,1336.1
4,id1598245,381.5
